#Import des bibliothèques

In [ ]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

import seaborn as sns
import matplotlib.pyplot as plt

#Import de la DB

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_pokemon = pd.read_csv('/content/drive/MyDrive/Python/Machine learning/pokemon.csv')

In [ ]:
df_pokemon.sample(8)

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
519,520,Togekiss,Fairy,Flying,85,50,95,120,115,80,4,False
773,774,Carbink,Rock,Fairy,50,50,150,50,150,50,6,False
240,241,Corsola,Water,Rock,55,55,85,65,85,35,2,False
652,653,Amoonguss,Grass,Poison,114,85,70,85,80,30,5,False
141,142,Mega Gyarados,Water,Dark,95,155,109,70,130,81,1,False
253,254,Stantler,Normal,NaN,73,95,62,85,65,85,2,False
513,514,Magnezone,Electric,Steel,70,70,115,130,90,60,4,False
186,187,Pichu,Electric,NaN,20,40,15,35,35,60,2,False


#Etude de la DB source

In [ ]:
df_pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        799 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   HP          800 non-null    int64 
 5   Attack      800 non-null    int64 
 6   Defense     800 non-null    int64 
 7   Sp. Atk     800 non-null    int64 
 8   Sp. Def     800 non-null    int64 
 9   Speed       800 non-null    int64 
 10  Generation  800 non-null    int64 
 11  Legendary   800 non-null    bool  
dtypes: bool(1), int64(8), object(3)
memory usage: 69.7+ KB


In [ ]:
df_pokemon.describe(include='all')

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
count,800.0000,799,800,414,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000,800
unique,NaN,799,18,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
top,NaN,Linoone,Water,Flying,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
freq,NaN,1,112,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,735
mean,400.5000,NaN,NaN,NaN,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,3.32375,NaN
std,231.0844,NaN,NaN,NaN,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,1.66129,NaN
min,1.0000,NaN,NaN,NaN,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.00000,NaN
25%,200.7500,NaN,NaN,NaN,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,2.00000,NaN
50%,400.5000,NaN,NaN,NaN,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,3.00000,NaN
75%,600.2500,NaN,NaN,NaN,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,5.00000,NaN


In [ ]:
df_pokemon.corr()

,#,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,1.000000,0.097712,0.102664,0.094691,0.089199,0.085596,0.012181,0.983428,0.154336
HP,0.097712,1.000000,0.422386,0.239622,0.362380,0.378718,0.175952,0.058683,0.273620
Attack,0.102664,0.422386,1.000000,0.438687,0.396362,0.263990,0.381240,0.051451,0.345408
Defense,0.094691,0.239622,0.438687,1.000000,0.223549,0.510747,0.015227,0.042419,0.246377
Sp. Atk,0.089199,0.362380,0.396362,0.223549,1.000000,0.506121,0.473018,0.036437,0.448907
Sp. Def,0.085596,0.378718,0.263990,0.510747,0.506121,1.000000,0.259133,0.028486,0.363937
Speed,0.012181,0.175952,0.381240,0.015227,0.473018,0.259133,1.000000,-0.023121,0.326715
Generation,0.983428,0.058683,0.051451,0.042419,0.036437,0.028486,-0.023121,1.000000,0.079794
Legendary,0.154336,0.273620,0.345408,0.246377,0.448907,0.363937,0.326715,0.079794,1.000000


In [ ]:
df_pokemon['Legendary'].sum()
#There are 65 legendary pokemon

65

In [ ]:
df_pokemon['Type 1'].value_counts()

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Rock         44
Electric     44
Ground       32
Ghost        32
Dragon       32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

In [ ]:
len(df_pokemon['Type 1'].value_counts())
#18 catégories de type 1

18

In [ ]:
len(df_pokemon['Type 1'][df_pokemon['Legendary'] == True].value_counts())
#15 catégories ont des pokemons Légendaires

15

In [ ]:
df_pokemon['Type 2'].value_counts()

Flying      97
Ground      35
Poison      34
Psychic     33
Fighting    26
Grass       25
Fairy       23
Steel       22
Dark        20
Dragon      18
Ghost       14
Ice         14
Rock        14
Water       14
Fire        12
Electric     6
Normal       4
Bug          3
Name: Type 2, dtype: int64

#Traitement des données

Type 2 : transformer en 0 ou 1 -> get dummies car plus de 2 références

HP,	Attack,	Defense,	Sp. Atk,	Sp. Def,	Speed -> standardisation des données

Generation : transformer en 0 ou 1 -> get dummies car plus de 2 références

Legendary : transformer en 0 ou 1 -> True = 1 et False = 0

In [ ]:
df_pokemon_c = df_pokemon.dropna(subset=['Name'])

In [ ]:
df_pokemon_c.reset_index(inplace=True)

In [ ]:
df_pokemon_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799 entries, 0 to 798
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index       799 non-null    int64 
 1   #           799 non-null    int64 
 2   Name        799 non-null    object
 3   Type 1      799 non-null    object
 4   Type 2      414 non-null    object
 5   HP          799 non-null    int64 
 6   Attack      799 non-null    int64 
 7   Defense     799 non-null    int64 
 8   Sp. Atk     799 non-null    int64 
 9   Sp. Def     799 non-null    int64 
 10  Speed       799 non-null    int64 
 11  Generation  799 non-null    int64 
 12  Legendary   799 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.8+ KB


In [ ]:
# #Type 1 : transformer en 0 ou 1 = devient notre cible (y)
df_type1 = df_pokemon_c['Type 1'].str.get_dummies()

In [ ]:
#df_type1.info()

In [ ]:
# #Type 2 : transformer en 0 ou 1 -> get dummies car plus de 2 références
df_type2 = df_pokemon_c['Type 2'].str.get_dummies()

In [ ]:
columns_name = df_type2.columns

In [ ]:
for name in columns_name :
  df_type2.rename(columns={name : 'T2_' + name},inplace=True)

In [ ]:
#df_type2.info()

In [ ]:
# #HP	Attack	Defense	Sp. Atk	Sp. Def	Speed -> standardisation des données
nb_car = ['HP',	'Attack',	'Defense',	'Sp. Atk', 'Sp. Def', 'Speed']
X_nb_car = df_pokemon_c[nb_car]

In [ ]:
scaler = StandardScaler().fit(X_nb_car)

In [ ]:
X_nb_car_scaled = scaler.transform(X_nb_car)

In [ ]:
df_nb_car = pd.DataFrame(X_nb_car_scaled,columns=nb_car)

In [ ]:
#df_nb_car.info()

In [ ]:
# #Generation : transformer en 0 ou 1 -> get dummies car plus de 2 références
df_generation = df_pokemon_c['Generation'].astype('str').str.get_dummies()

In [ ]:
columns_name = df_generation.columns

In [ ]:
for name in columns_name :
  df_generation.rename(columns={name : 'Generation_' + name},inplace=True)

In [ ]:
#df_generation

In [ ]:
# #Legendary : transformer en 0 ou 1 -> True = 1 et False = 0
df_legendary = df_pokemon_c['Legendary'].apply(lambda x : 1 if x == True else 0)

In [ ]:
#df_legendary

In [ ]:
df_pokemon_AL = pd.concat([df_pokemon_c['Name'],df_type1, df_type2, df_nb_car, df_generation, df_legendary],axis=1)

In [ ]:
df_pokemon_AL.head()

,Name,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water,T2_Bug,T2_Dark,T2_Dragon,T2_Electric,T2_Fairy,T2_Fighting,T2_Fire,T2_Flying,T2_Ghost,T2_Grass,T2_Ground,T2_Ice,T2_Normal,T2_Poison,T2_Psychic,T2_Rock,T2_Steel,T2_Water,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation_1,Generation_2,Generation_3,Generation_4,Generation_5,Generation_6,Legendary
0,Bulbasaur,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-0.950257,-0.923697,-0.797309,-0.239494,-0.248120,-0.800275,1,0,0,0,0,0,0
1,Ivysaur,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-0.362810,-0.523010,-0.348298,0.218953,0.290890,-0.283837,1,0,0,0,0,0,0
2,Venusaur,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.420452,0.093431,0.293146,0.830216,1.009569,0.404748,1,0,0,0,0,0,0
3,Mega Venusaur,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0.420452,0.648227,1.576034,1.502605,1.728248,0.404748,1,0,0,0,0,0,0
4,Charmander,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1.185236,-0.831231,-0.989742,-0.392310,-0.787129,-0.111690,1,0,0,0,0,0,0


#Création du modèle Knn sans les légendaires

Je retire les légendaires du modèle car ils ne doivent pas ressortir dans les propositions de mon modèle

In [ ]:
df_pokemon_AL_without_lengendary = df_pokemon_AL[df_pokemon_AL['Legendary'] == 0]
df_pokemon_AL_without_lengendary.drop(columns='Legendary',inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#df_pokemon_AL_without_lengendary.info()

In [ ]:
X_list = list(df_pokemon_AL_without_lengendary.columns)
X_list.remove('Name')

In [ ]:
#X_list

In [ ]:
X = df_pokemon_AL_without_lengendary[X_list]
y = df_pokemon_AL_without_lengendary['Name']

In [ ]:
#je cherche le pokemon le plus proche, je ne cherche pas à classifier en tant que tel mon pokemon.
##j'ai donc seulement besoin du plus proche voisin
model_KNN = KNeighborsClassifier(n_neighbors=1,weights='distance')

In [ ]:
model_KNN.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='distance')

In [ ]:
def substitut_pokemon(poke_source):
  X_poke_source = df_pokemon_AL[X_list][df_pokemon_AL['Name'] == poke_source]
  list_X_poke_source = list(X_poke_source.iloc[0])
  distance, classement = model_KNN.kneighbors([list_X_poke_source])
  classement_condition =  classement[0]
  df_classement = df_pokemon_AL_without_lengendary.iloc[classement_condition]
  df_classement.reset_index(drop=True, inplace=True)
  df_classement['Classement'] = [1] #10,9,8,7,6,5,4,3, 2,
  pokemon_substitut = df_classement['Name'].iloc[0]
  return pokemon_substitut

In [ ]:
substitut_pokemon('Lugia')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


'Cresselia'

In [ ]:
print('The best subsitute for the lengendary pokemon', pokemon, 'is', substitut_pokemon('Lugia'), '.')

The best subsitute for the lengendary pokemon Palkia is Cresselia .


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
list_replace_pokemon = ['Mewtwo', 'Lugia', 'Rayquaza', 'Giratina Altered Forme', 'Giratina Origin Forme', 'Dialga', 'Palkia']

In [ ]:
#substitut_pokemon('Dialga')

In [ ]:
for pokemon in list_replace_pokemon :
  print('The best subsitute for the lengendary pokemon', pokemon, 'is', substitut_pokemon(pokemon), '.')
print()

The best subsitute for the lengendary pokemon Mewtwo is Mew .
The best subsitute for the lengendary pokemon Lugia is Cresselia .
The best subsitute for the lengendary pokemon Rayquaza is Salamence .
The best subsitute for the lengendary pokemon Giratina Altered Forme is Cresselia .
The best subsitute for the lengendary pokemon Giratina Origin Forme is Manaphy .
The best subsitute for the lengendary pokemon Dialga is Mega Charizard X .
The best subsitute for the lengendary pokemon Palkia is Manaphy .



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s